In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [2]:
import src.load as load
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import torch.nn as nn
from torch.nn.utils import weight_norm

In [37]:
meta, y, schedule, mappings, reverse_mappings = load.load_data('../data/processed/')

In [40]:
meta

,TransplantDate,Parcel,Lot,ProducerCode,Variety,Ha,Class,Type,WeekTransplanted,WeekCos,WeekSin,Year,ClimateSeries
0,2012-03-28,N02,7,OAP,Shiren,0.1330,CHE,Cherry Rojo,13,-1.608123e-16,1.000000,2012,"[[-0.3827101525, -1.1992770129, 0.0, 0.0, 0.41..."
1,2013-02-13,Z18,6,ZJL,Felicity,0.3938,CHE,Cherry Rojo,7,6.631227e-01,0.748511,2013,"[[-0.2535675497, -0.8592687109, 0.0, 0.0, -0.1..."
2,2013-02-13,Z18,6,ZJL,Shiren,0.3938,CHE,Cherry Rojo,7,6.631227e-01,0.748511,2013,"[[-0.2535675497, -0.8592687109, 0.0, 0.0, -0.1..."
3,2013-02-15,Z18,2,ZJL,Amsterdam,0.2700,BSUF,Uva Roja,7,6.631227e-01,0.748511,2013,"[[0.3125368926, -0.0596579057, 0.0, 0.0, -0.54..."
4,2013-02-15,Z18,2,ZJL,Olivia,0.5400,BSUF,Uva Roja,7,6.631227e-01,0.748511,2013,"[[0.3125368926, -0.0596579057, 0.0, 0.0, -0.54..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3785,2024-07-25,S12,4,SGB,Top 2323,0.1530,CHE,Cherry Rosa,30,-8.854560e-01,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3786,2024-07-25,S12,4,SGB,Yoyomo,0.1020,BSUF,Uva Amarilla,30,-8.854560e-01,-0.464723,2024,"[[0.8452649894, 0.93700445, 0.0, 0.0, 0.485896..."
3787,2024-07-29,S12,7,SGB,King,0.5100,BSUF,Uva Roja,31,-8.229839e-01,-0.568065,2024,"[[0.3676142667, 0.3320816298, 0.0, 0.0, 0.5941..."
3788,2024-07-29,S12,8,SGB,King,0.0306,BSUF,Uva Roja,31,-8.229839e-01,-0.568065,2024,"[[0.3676142667, 0.3320816298, 0.0, 0.0, 0.5941..."


In [41]:
y

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3786,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,88.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
schedule.shape

(3790, 5)

In [35]:
y.shape

(3790, 40)

In [5]:
kilo_dist = (y.values / y.values.sum(axis=1, keepdims=True)).cumsum(axis=1)

In [6]:
y.sum(axis=1)

0       14768.0
1       21302.0
2       24530.0
3        7232.0
4       10464.0
         ...   
3785     1000.0
3786      396.0
3787     7789.0
3788     2267.0
3789     9736.0
Length: 3790, dtype: float64

In [7]:
class Data_Take2(Dataset):


    def __init__(self, 
                 features,
                 encoded_features,
                 climate_data,
                 yield_dist,       
                 kilo_dist,
                 yield_log,       
                 schedule
                ):

        self.features = torch.tensor(features, dtype=torch.float32)
        self.encoded_features = torch.tensor(encoded_features, dtype=torch.long)
        self.climate_data = torch.tensor(climate_data, dtype=torch.float32)
        self.yield_dist = torch.tensor(yield_dist, dtype=torch.float32).unsqueeze(2)
        self.kilo_dist = torch.tensor(kilo_dist, dtype=torch.float32).unsqueeze(2)

        self.Y_yield_log = torch.tensor(yield_log, dtype=torch.float32)
        self.Y_schedule = torch.tensor(schedule, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        climate_data = self.climate_data[idx]

        return (
            self.features[idx],
            self.encoded_features[idx],
            climate_data,
            self.yield_dist[idx],
            self.kilo_dist[idx],
            self.Y_yield_log[idx],
            self.Y_schedule[idx],
            idx
        )
    
    def get_shapes(self):

        shapes = {
            'features': self.features.shape,
            'encoded_features': self.encoded_features.shape,
            'climate_data': self.climate_data.shape,
            'yield_dist': self.yield_dist.shape,
            'kilo_dist': self.kilo_dist.shape,
            'yield_log': self.Y_yield_log.shape,
            'schedule': self.Y_schedule.shape
        }
        return shapes

In [44]:
features = np.column_stack([
    meta['Ha'].to_numpy(),  # Hectares
    meta['WeekSin'].to_numpy(),  # Week sine
    meta['WeekCos'].to_numpy(),  # Week cosine
    meta['Year'].to_numpy() - 2010,                  # Year
    np.ones(len(meta))                    # Constant feature
])
mapped_arrays = [meta[column].astype(str).map(mappings[column]).to_numpy() for column in ['ProducerCode','Parcel','Class','Type','Variety']]
encoded_features = np.column_stack(mapped_arrays)

climate_data = np.array(meta.ClimateSeries.to_list())
schedule_data = schedule.values
kilo_dist = (y.to_numpy() / y.to_numpy().sum(axis=1, keepdims=True)).cumsum(axis=1)
yield_dist = np.log1p(y.to_numpy() / meta['Ha'].to_numpy()[:, np.newaxis])
yield_log = np.log1p(y.to_numpy().sum(axis=1) / meta['Ha'].to_numpy())

data_set = Data_Take2(
    features,
    encoded_features,
    climate_data,
    yield_dist,
    kilo_dist,
    yield_log,
    schedule_data
)

In [9]:
data_set.kilo_dist.shape

torch.Size([3790, 40, 1])

In [10]:
class ClimateEncoder(nn.Module):
    def __init__(self,
                 input_dim=5,
                 embedding_dim=12,
                 hidden_dim=32,
                 n_ranches=13,
                 n_parcels=44,
                 n_classes=2,
                 n_types=14,
                 n_varieties=59,
                 climate_input_dim=9,
                 climate_hidden_dim=32):
        super().__init__()

        # Feature processing
        self.feature_encoder = nn.Sequential(
            weight_norm(nn.Linear(input_dim, hidden_dim)),
            nn.ReLU()
        )

        # Climate GRU
        self.climate_gru = nn.GRU(
            input_size=climate_input_dim, 
            hidden_size=climate_hidden_dim,
            num_layers=2,
            dropout=0.2,
            batch_first=True
        )

        # Embedding dimensions
        self.ranch_dim = embedding_dim  # 12 ranches
        self.parcel_dim = embedding_dim  # 12 ranches
        self.class_dim = embedding_dim  # 2 classes
        self.type_dim = embedding_dim  # 14 types
        self.variety_dim = embedding_dim  # 38 varieties

        self.ranch_emb = nn.Embedding(n_ranches, self.ranch_dim)
        self.parcel_emb = nn.Embedding(n_parcels, self.parcel_dim)

        self.parcel_to_ranch = weight_norm(nn.Linear(self.parcel_dim, self.ranch_dim))

        self.class_emb = nn.Embedding(n_classes, self.class_dim)
        self.type_emb = nn.Embedding(n_types, self.type_dim)
        self.variety_emb = nn.Embedding(n_varieties, self.variety_dim)

        self.type_to_class = weight_norm(nn.Linear(self.type_dim, self.class_dim))
        self.variety_to_type = weight_norm(nn.Linear(self.variety_dim, self.type_dim))

        self.combined_dim = (
            hidden_dim +             
            climate_hidden_dim +     
            self.ranch_dim + 
            self.parcel_dim + 
            self.class_dim + 
            self.type_dim + 
            self.variety_dim
        )


    def forward(self, features, encoded_features, climate_data):
        """
        features: (batch_size, 5)
        climate_data: (batch_size, 100, )
        """

        # Static feature encoder
        h_features = self.feature_encoder(features)

        # Climate GRU
        batch_size = climate_data.size(0)
        h0 = torch.zeros(2, batch_size, self.climate_gru.hidden_size).to(climate_data.device)
        out, _ = self.climate_gru(climate_data, h0)  # out: (batch_size, seq_len, hidden_size)

        # Take last timestep
        climate_out = out[:, -1, :]  # (batch_size, climate_hidden_dim)

        ranch_id, parcel_id, class_id, type_id, variety_id = encoded_features.T

        # Embeddings
        r_emb = self.ranch_emb(ranch_id)
        p_emb = self.parcel_emb(parcel_id)

        p_influence_on_ranch = self.parcel_to_ranch(p_emb)
        r_emb = r_emb + p_influence_on_ranch

        c_emb = self.class_emb(class_id)
        t_emb = self.type_emb(type_id)
        v_emb = self.variety_emb(variety_id)

        # Hierarchy
        v_influence_on_type = self.variety_to_type(v_emb)
        t_emb = t_emb + v_influence_on_type

        t_influence_on_class = self.type_to_class(t_emb)
        c_emb = c_emb + t_influence_on_class

        # Combine all features
        combined = torch.cat([
            h_features,
            climate_out,
            r_emb,
            p_emb,
            c_emb,
            t_emb,
            v_emb
        ], dim=-1)

        return combined

In [11]:
class DistModel(nn.Module):
    def __init__(self,
                 output_dim=40,
                 hidden_size=32,
                 batch_size=32):
        super().__init__()
        
        self.output_dim = output_dim
        
        self.encoder = ClimateEncoder()

        self.kilo_gru = nn.GRU(
            input_size=1,
            num_layers=2,
            dropout=0.2,
            hidden_size=hidden_size,
            batch_first=True
        )

        self.kilo_output = nn.Sequential(
            weight_norm(nn.Linear(self.encoder.combined_dim+hidden_size, batch_size)),
            nn.ReLU(),
            weight_norm(nn.Linear(batch_size, output_dim))
        )

    def forward(self, features, encoded_features, climate_data, kilo_gru_input):
        """
        features: (batch_size, 5)
        climate_data: (batch_size, 100, 10)
        kilo_dist_input: (batch_size, 40, 2)
        """
        encoded = self.encoder(features, encoded_features, climate_data)
        batch_size = climate_data.size(0)
        
        # Ensure h0 is on the same device as the input
        h0 = torch.zeros(2, batch_size, self.kilo_gru.hidden_size, device=kilo_gru_input.device)
        out, _ = self.kilo_gru(kilo_gru_input, h0)
        result = out[:, -1, :]
    
        kilo_output = self.kilo_output(torch.cat((encoded,result),dim=1))
        return kilo_output

In [31]:
class ScheduleModel(nn.Module):
    def __init__(self,
                 output_dim=5,
                 batch_size=32,
                 hidden_size=32):
        super().__init__()
        
        self.output_dim = output_dim
        self.encoder = ClimateEncoder()

        self.kilo_gru = nn.GRU(
            input_size=1,
            hidden_size=hidden_size,
            num_layers=2,
            dropout=0.2,
            batch_first=True
        )

        self.schedule_output = nn.Sequential(
            weight_norm(nn.Linear(self.encoder.combined_dim+hidden_size, batch_size)),
            nn.ReLU(),
            weight_norm(nn.Linear(batch_size, output_dim))
        )

    def forward(self, features, encoded_features, climate_data, kilo_gru_input):
        """
        features: (batch_size, 5)
        climate_data: (batch_size, 100, 10)
        kilo_gru_input: (batch_size, [5:20], 3)
        kilo_model_output: (batch_size, 40)
        """
        encoded = self.encoder(features, encoded_features, climate_data)
        batch_size = climate_data.size(0)
        
        # Ensure h0 is on the same device as the input
        h0 = torch.zeros(2, batch_size, self.kilo_gru.hidden_size, device=kilo_gru_input.device)
        out, _ = self.kilo_gru(kilo_gru_input, h0)
        result = out[:, -1, :]
    
        schedule = self.schedule_output(torch.cat((encoded,result),dim=1))
        return schedule
    


In [13]:
class YieldModel(nn.Module):
    def __init__(self,
                 batch_size=32,
                 hidden_size=32):
        super().__init__()

        self.encoder = ClimateEncoder()
        self.yield_gru = nn.GRU(
            input_size=1,
            hidden_size=32,
            num_layers=2,
            dropout=0.2,
            batch_first=True
        )
        
        self.final_output = nn.Sequential(
            weight_norm(nn.Linear(self.encoder.combined_dim+hidden_size, batch_size)),
            nn.ReLU(),
            weight_norm(nn.Linear(batch_size, 1))
        )
        
    def forward(self, features, encoded_features, climate_data, yield_gru_input):
        encoded = self.encoder(features, encoded_features, climate_data)
        batch_size = climate_data.size(0)
        h0 = torch.zeros(2, batch_size, self.yield_gru.hidden_size)
        out, _ = self.yield_gru(yield_gru_input, h0)
        result = out[:, -1, :]
        yield_output = self.final_output(torch.cat((encoded, result), dim=1))
        return yield_output

In [14]:
data_set.get_shapes()

{'features': torch.Size([3790, 5]),
 'encoded_features': torch.Size([3790, 5]),
 'climate_data': torch.Size([3790, 100, 9]),
 'yield_dist': torch.Size([3790, 40, 1]),
 'kilo_dist': torch.Size([3790, 40, 1]),
 'yield_log': torch.Size([3790]),
 'schedule': torch.Size([3790, 5])}

In [23]:
from torch.utils.data import DataLoader

dataloader = DataLoader(data_set, batch_size=32, shuffle=True)
model = YieldModel()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-4)
criterion = nn.MSELoss()

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

/Users/zakir/anaconda3/envs/cherry/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [24]:
num_weeks = 5

In [25]:
for epoch in range(10):
    total_loss = 0
    for batch in dataloader:
        features, encoded_features, climate_data, yield_dist, kilo_dist, yield_log, schedule,_ = batch
        
        yield_gru_input = yield_dist[:,:num_weeks,:]
        yield_output = model(features, encoded_features, climate_data, yield_gru_input)
        yield_output = yield_output.squeeze(-1)
        loss = criterion(torch.exp(yield_output), torch.exp(yield_log))

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")


Epoch 1, Loss: 793817033.0084034
Epoch 2, Loss: 411957184.4033613
Epoch 3, Loss: 343710810.68907565
Epoch 4, Loss: 327882487.39495796
Epoch 5, Loss: 327370522.8907563
Epoch 6, Loss: 314703600.26890755
Epoch 7, Loss: 303755938.75630254
Epoch 8, Loss: 293399165.4453781
Epoch 9, Loss: 304933595.0252101
Epoch 10, Loss: 291552681.5462185


In [26]:
dist_model = DistModel()

dist_optimizer = torch.optim.Adam(dist_model.parameters(), lr=1e-3,weight_decay=1e-4)

dist_scheduler = torch.optim.lr_scheduler.StepLR(dist_optimizer, step_size=10, gamma=0.1)

/Users/zakir/anaconda3/envs/cherry/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [30]:
for epoch in range(10):
    total_loss = 0
    for batch in dataloader:
        features, encoded_features, climate_data, yield_dist, kilo_dist, yield_log, schedule,_ = batch
        
        kilo_gru_input = kilo_dist[:,:num_weeks,:]
        dist_output = dist_model(features, encoded_features, climate_data, kilo_gru_input)
        dist_output = dist_output.squeeze(-1)
        loss = criterion(dist_output, kilo_dist.squeeze(-1))

        torch.nn.utils.clip_grad_norm_(dist_model.parameters(), max_norm=1.0)

        dist_optimizer.zero_grad()
        loss.backward()
        dist_optimizer.step()
        total_loss += loss.item()
    dist_scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch 1, Loss: 0.09499948550829616
Epoch 2, Loss: 0.016040221631464336
Epoch 3, Loss: 0.015300306984606911
Epoch 4, Loss: 0.0149829226757298
Epoch 5, Loss: 0.014755141389157091
Epoch 6, Loss: 0.014639073935877375
Epoch 7, Loss: 0.014281494284690428
Epoch 8, Loss: 0.013998339218752725
Epoch 9, Loss: 0.013788929506510245
Epoch 10, Loss: 0.013617030556211952


In [32]:
sched_model = ScheduleModel()

sched_optimizer = torch.optim.Adam(sched_model.parameters(), lr=1e-3,weight_decay=1e-4)

sched_scheduler = torch.optim.lr_scheduler.StepLR(sched_optimizer, step_size=10, gamma=0.1)

/Users/zakir/anaconda3/envs/cherry/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [33]:
for epoch in range(10):
    total_loss = 0
    for batch in dataloader:
        features, encoded_features, climate_data, yield_dist, kilo_dist, yield_log, schedule,_ = batch
        
        kilo_gru_input = kilo_dist[:,:num_weeks,:]
        sched_output = sched_model(features, encoded_features, climate_data, kilo_gru_input)
        sched_output = sched_output.squeeze(-1)
        loss = criterion(sched_output, schedule)

        torch.nn.utils.clip_grad_norm_(sched_model.parameters(), max_norm=1.0)

        sched_optimizer.zero_grad()
        loss.backward()
        sched_optimizer.step()
        total_loss += loss.item()
    sched_scheduler.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch 1, Loss: 53.16181524861761
Epoch 2, Loss: 5.803398048176485
Epoch 3, Loss: 5.222241886523592
Epoch 4, Loss: 4.987875635884389
Epoch 5, Loss: 4.851659502301898
Epoch 6, Loss: 4.774711628921893
Epoch 7, Loss: 4.683125724311636
Epoch 8, Loss: 4.622283482751927
Epoch 9, Loss: 4.578349255714096
Epoch 10, Loss: 4.53223679246021
